Notebook to obtain SISTR adjusted p values after running SISTR

In [7]:
import statsmodels.api
from statsmodels.stats import multitest

In [8]:
def main():  
    print('Running main')
    num_dinuc = 0
    num_trinuc = 0
    num_tetranuc = 0
    combos = [('2','50k'),('2','eurodem'),('3','eurodem'),('2','20k'),('2','50k')]
    for combo in combos:
        
        p_val_list = []
        prior = combo[0] 
        model = combo[1] 
        print(prior + ' ' + model)
        total_threshold = 5214
        inFile = '/storage/BonnieH/selection_project/per_locus/SISTR_results/0.005_3_0.3_yny5_%s_prior%s_all_per_0810.txt'%(model, prior) 
        in_file = open(inFile, 'r')
        header = in_file.readline().strip()

        outFile = '/storage/BonnieH/selection_project/per_locus/SISTR_results_processed/0.005_3_0.3_yny5_%s_prior%s_all_per_0810.txt'%(model, prior)
        out_file = open(outFile, 'w')
        
        for line in in_file:

            # Get information from line
            info = line.strip().split('\t')
            chrom = info[0]
            start = int(info[1])
            end = int(info[2])
            per = int(info[4])
            optimal_ru = int(info[5])
            ABC_s = info[17]
            total_n = int(info[3])

            if total_n > total_threshold:
                if per == 2:
                    num_dinuc = num_dinuc + 1
                if per == 3:
                    num_trinuc = num_trinuc + 1
                if per == 4:
                    num_tetranuc = num_tetranuc + 1
                    
            if ABC_s != 'N/A' and total_n > total_threshold: 
                
                p_val = float(info[24])
                p_val_list.append(p_val)

        in_file.close()
        adj_p_val = statsmodels.stats.multitest.fdrcorrection(p_val_list)
        
        out_file.write("chrom" + "\t" + "start" + "\t" + "end" + "\t" + "total" + "\t" + "period" + \
                  "\t" + "optimal_ru" + "\t" + "motif" + "\t" + "coding" + "\t"  + 'intron' + \
                  '\t' + 'UTR5' + '\t' + 'UTR3' + '\t' + 'promoter5kb' + '\t' + 'intergenic' + \
                  "\t" + "tss_gene" + "\t" + "het" + "\t" + "common" + "\t" + "bins" + "\t" + \
                  "ABC_s_median" + "\t" + "ABC_s_95%_CI" + "\t" + "Num_s_accepted_(max_10000)" + "\t" + \
                  "Likelihood_0" + "\t" + "Likelihood_s" + "\t" + "LR" + "\t" + "LogLR" + \
                  "\t" + "LRT_p_value" + "\t" + "Adjusted_p_value" + "\n")
        
        in_file = open(inFile, 'r')
        header = in_file.readline().strip()
        i = 0
        for line in in_file:
            info = line.strip().split('\t')
            chrom = info[0]
            start = int(info[1])
            end = int(info[2])
            per = int(info[4])
            optimal_ru = int(info[5])
            ABC_s = info[17]
            total_n = int(info[3])

            if ABC_s != 'N/A' and total_n > total_threshold: 
               
                line = line.strip('\n')
                out_file.write(line)
                out_file.write('\t' + str(adj_p_val[1][i]) + '\n')
                i = i + 1
        in_file.close()
        out_file.close()
    print('Number of dinucleotides: ' + str(num_dinuc))
    print('Number of trinucleotides: ' + str(num_trinuc))
    print('Number of tetranucleotides: ' + str(num_tetranuc))

In [9]:
%%time
if __name__ == '__main__':
    main()

Running main
2 50k
Number of dinucleotides: 29874
Number of trinucleotides: 39250
Number of tetranucleotides: 13099
CPU times: user 1.83 s, sys: 268 ms, total: 2.1 s
Wall time: 1.92 s
